# Análisis de Flujos de Dinero en Ekhilur

Este notebook realiza un análisis detallado de las transacciones financieras en la plataforma Ekhilur, utilizando consultas directas a la base de datos MySQL.

## 1. Configuración Inicial

Primero importamos las librerías necesarias y establecemos la conexión con la base de datos.

In [13]:
# Importar librerías necesarias
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette('husl')
%matplotlib inline

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\mikel\AppData\Local\Temp\ipykernel_41176\1802020149.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


### Función para conectar a la base de datos

Creamos una función helper para manejar las consultas SQL.

In [14]:
import os
from dotenv import load_dotenv
import mysql.connector
import pandas as pd

# Cargar variables de entorno
load_dotenv()

def get_db_connection():
    return mysql.connector.connect(
        host='localhost',  # Usando localhost porque estamos fuera del contenedor
        port=3308,        # Puerto mapeado en docker-compose
        user='user',      # Usuario definido en docker-compose
        password='userpassword',  # Password definido en docker-compose
        database='ekhilur'  # Base de datos definida en docker-compose
    )

def ejecutar_query(query):
    """Ejecuta una query SQL y devuelve los resultados como DataFrame"""
    conn = get_db_connection()
    try:
        return pd.read_sql_query(query, conn)
    finally:
        conn.close()

## 2. Análisis de Tipos de Operaciones

Analizamos la distribución de las operaciones y sus importes asociados.

In [15]:
# Consulta para obtener los tipos de operaciones y sus estadísticas
query_operaciones = """
SELECT 
    o.Operacion,
    COUNT(*) as total_operaciones,
    SUM(f.Importe) as suma_total,
    AVG(f.Importe) as promedio_importe
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY o.Operacion
ORDER BY suma_total DESC;
"""

df_operaciones = ejecutar_query(query_operaciones)

# Crear gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(name='Suma Total', x=df_operaciones['Operacion'], y=df_operaciones['suma_total'])
])

fig.update_layout(
    title='Volumen Total por Tipo de Operación',
    xaxis_title='Tipo de Operación',
    yaxis_title='Suma Total (€)',
    template='plotly_white'
)

fig.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_41176\3686747407.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


DatabaseError: Execution failed on sql '
SELECT 
    o.Operacion,
    COUNT(*) as total_operaciones,
    SUM(f.Importe) as suma_total,
    AVG(f.Importe) as promedio_importe
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY o.Operacion
ORDER BY suma_total DESC;
': 1054 (42S22): Unknown column 'f.Importe' in 'field list'

## 3. Análisis Temporal de Transacciones

Analizamos cómo evolucionan las transacciones a lo largo del tiempo.

In [ ]:
# Consulta para análisis temporal
query_temporal = """
SELECT 
    DATE(CONCAT(
        SUBSTRING(f.Id_fecha, 1, 4), '-',
        SUBSTRING(f.Id_fecha, 5, 2), '-',
        SUBSTRING(f.Id_fecha, 7, 2)
    )) as fecha,
    o.Operacion,
    SUM(f.Importe) as suma_diaria
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY fecha, o.Operacion
ORDER BY fecha;
"""

df_temporal = ejecutar_query(query_temporal)

# Crear gráfico de líneas temporal
fig = px.line(df_temporal, 
              x='fecha', 
              y='suma_diaria', 
              color='Operacion',
              title='Evolución Temporal de Transacciones por Tipo')

fig.update_layout(
    xaxis_title='Fecha',
    yaxis_title='Importe Total Diario (€)',
    template='plotly_white'
)

fig.show()